In [1]:
import json
import pickle
import os
import math
import multiprocessing as mp
import random
from random import shuffle, seed
import re

seed(123)

### Get a list of Files

In [3]:
filenames = os.listdir('/data/liufengyuan/NLPinFinance/Unziped_Filtered_Data')
filenames

['Cell_Phones_and_Accessories.pkl',
 'Office_Products.pkl',
 'Movies_and_TV.pkl',
 'Patio_Lawn_and_Garden.pkl',
 'Clothing_Shoes_and_Jewelry.pkl',
 'Electronics.pkl',
 'Industrial_and_Scientific.pkl',
 'Kindle_Store.pkl',
 'Digital_Music.pkl',
 'Home_and_Kitchen.pkl',
 'Sports_and_Outdoors.pkl',
 'CDs_and_Vinyl.pkl',
 'Books.pkl',
 'Arts_Crafts_and_Sewing.pkl',
 'Toys_and_Games.pkl',
 'Grocery_and_Gourmet_Food.pkl',
 'AMAZON_FASHION.pkl',
 'Pet_Supplies.pkl',
 'Automotive.pkl',
 'Tools_and_Home_Improvement.pkl']

In [14]:
f_read = open('/data/liufengyuan/NLPinFinance/Unziped_Filtered_Data/Clothing_Shoes_and_Jewelry.pkl', 'rb')
All_data = pickle.load(f_read)
f_read.close()

In [21]:
All_data[3]['description']

["Pantone Accuracy and Convenience for Four-Color Printing. The new PLUS SERIES CMYK guides contain 2,868 colors, chromatically arranged for smoother transitions between colors and more intuitive selection. These guides are an ideal way to visualize, communicate and control colors for type, logos, borders, backgrounds and other graphics treatments. Compatible with today's digital workflows. Screen tint percentages are provided to enable accurate color reproduction. Printed using bio-friendly, ISO-certified inks. Text weight paper is used, to match popular print specifications. Includes color index, lighting evaluation tool and design software. Key product benefits: 1) Accurately select, specify and communicate color for four-color process printing; 2) Easy location of desired colors; 3) Easily reproduce colors in process printing with supplied CMYK screen percentages; 4) Shows the effect of coated and uncoated stocks on each color; 5) Provides a printed standard for color comparison an

In [15]:
file_dict = {}

In [16]:
for file in filenames:
    f_read = open('/data/liufengyuan/NLPinFinance/Unziped_Filtered_Data/' + file, 'rb')
    All_data = pickle.load(f_read)
    f_read.close()
    temp_dict = {}

    tot_cap_len = 0
    tot_fea_len = 0
    tot_des_len = 0
    for idx, item in enumerate(All_data):
        tot_cap_len += len(item['title'])
        for fea in item['feature']:
            tot_fea_len += len(fea)
        for fea in item['description']:
            tot_des_len += len(fea)
    print(f"{file}: (total length: {len(All_data)}) has mean title length {tot_cap_len/(1+len(All_data))}, mean feature length {tot_fea_len/(1+len(All_data))}, mean description length {tot_des_len/(1+len(All_data))}")
    temp_dict['length'] = len(All_data)
    temp_dict['title'] = tot_cap_len/(1+len(All_data))
    temp_dict['feature'] = tot_fea_len/(1+len(All_data))
    temp_dict['description'] = tot_des_len/(1+len(All_data))
    file_dict[file[:-4]] = temp_dict
    

Cell_Phones_and_Accessories.pkl: (total length: 282198) has mean title length 129.80852518967112, mean feature length 472.4708875651579, mean description length 660.0881044936375
Office_Products.pkl: (total length: 135100) has mean title length 87.18961369641971, mean feature length 401.0041598507783, mean description length 580.8472994278354
Movies_and_TV.pkl: (total length: 53) has mean title length 63.148148148148145, mean feature length 338.037037037037, mean description length 901.7962962962963
Patio_Lawn_and_Garden.pkl: (total length: 112378) has mean title length 81.9882362363075, mean feature length 429.0063179063704, mean description length 667.7810178058178
Clothing_Shoes_and_Jewelry.pkl: (total length: 1513427) has mean title length 621.0240711814503, mean feature length 366.00089597919424, mean description length 516.8315830022967
Electronics.pkl: (total length: 420725) has mean title length 108.96757509638101, mean feature length 484.1350546436398, mean description length 

In [25]:
old_file_dict = file_dict.copy()

In [26]:
old_file_dict

{'Cell_Phones_and_Accessories': {'length': 282198,
  'title': 129.80852518967112,
  'feature': 472.4708875651579,
  'description': 660.0881044936375},
 'Office_Products': {'length': 135100,
  'title': 87.18961369641971,
  'feature': 401.0041598507783,
  'description': 580.8472994278354},
 'Movies_and_TV': {'length': 53,
  'title': 63.148148148148145,
  'feature': 338.037037037037,
  'description': 901.7962962962963},
 'Patio_Lawn_and_Garden': {'length': 112378,
  'title': 81.9882362363075,
  'feature': 429.0063179063704,
  'description': 667.7810178058178},
 'Clothing_Shoes_and_Jewelry': {'length': 1513427,
  'title': 621.0240711814503,
  'feature': 366.00089597919424,
  'description': 516.8315830022967},
 'Electronics': {'length': 420725,
  'title': 108.96757509638101,
  'feature': 484.1350546436398,
  'description': 880.4408498642822},
 'Industrial_and_Scientific': {'length': 25819,
  'title': 88.29918667699458,
  'feature': 394.8879163439194,
  'description': 713.4946553059643},
 'K

In [33]:
for file in filenames:
    f_read = open('/data/liufengyuan/NLPinFinance/Unziped_Filtered_Data/' + file, 'rb')
    All_data = pickle.load(f_read)
    f_read.close()

    count = 0

    for idx, item in enumerate(All_data):

        tot_cap_len = len(item['title'])

        if tot_cap_len < file_dict[file[:-4]]['title']:
            tot_fea_len = 0
            for fea in item['feature']:
                tot_fea_len += len(fea)
            
            if tot_fea_len < file_dict[file[:-4]]['feature']:
                tot_des_len = 0
                for des in item['description']:
                    tot_des_len += len(des)

                if tot_des_len < file_dict[file[:-4]]['description']:
                    count+=1
    file_dict[file[:-4]]['length'] = count

In [34]:
file_dict

{'Cell_Phones_and_Accessories': {'length': 95188,
  'title': 129.80852518967112,
  'feature': 472.4708875651579,
  'description': 660.0881044936375},
 'Office_Products': {'length': 50768,
  'title': 87.18961369641971,
  'feature': 401.0041598507783,
  'description': 580.8472994278354},
 'Movies_and_TV': {'length': 14,
  'title': 63.148148148148145,
  'feature': 338.037037037037,
  'description': 901.7962962962963},
 'Patio_Lawn_and_Garden': {'length': 38839,
  'title': 81.9882362363075,
  'feature': 429.0063179063704,
  'description': 667.7810178058178},
 'Clothing_Shoes_and_Jewelry': {'length': 768609,
  'title': 621.0240711814503,
  'feature': 366.00089597919424,
  'description': 516.8315830022967},
 'Electronics': {'length': 149181,
  'title': 108.96757509638101,
  'feature': 484.1350546436398,
  'description': 880.4408498642822},
 'Industrial_and_Scientific': {'length': 9302,
  'title': 88.29918667699458,
  'feature': 394.8879163439194,
  'description': 713.4946553059643},
 'Kindle

In [41]:
sorted(file_dict.items(), key = lambda x: x[1]['length'], reverse=True)

[('Clothing_Shoes_and_Jewelry',
  {'length': 768609,
   'title': 621.0240711814503,
   'feature': 366.00089597919424,
   'description': 516.8315830022967}),
 ('Home_and_Kitchen',
  {'length': 198095,
   'title': 83.04058980043519,
   'feature': 430.47090806370113,
   'description': 581.5831457887779}),
 ('Electronics',
  {'length': 149181,
   'title': 108.96757509638101,
   'feature': 484.1350546436398,
   'description': 880.4408498642822}),
 ('Automotive',
  {'length': 143482,
   'title': 78.71878514111359,
   'feature': 379.6169508197504,
   'description': 725.1025195357732}),
 ('Sports_and_Outdoors',
  {'length': 137210,
   'title': 68.03204798722673,
   'feature': 822.8115232773044,
   'description': 576.3706761737077}),
 ('Cell_Phones_and_Accessories',
  {'length': 95188,
   'title': 129.80852518967112,
   'feature': 472.4708875651579,
   'description': 660.0881044936375}),
 ('Tools_and_Home_Improvement',
  {'length': 84358,
   'title': 87.55753411318422,
   'feature': 403.7027506

### Get top five files

* 'Clothing_Shoes_and_Jewelry',
* 'Home_and_Kitchen',
* 'Electronics',
* 'Automotive',
* 'Sports_and_Outdoors'

In [2]:
new_file_name = ['Clothing_Shoes_and_Jewelry.pkl', 'Home_and_Kitchen.pkl', 'Electronics.pkl', 'Automotive.pkl', 'Sports_and_Outdoors.pkl']

In [46]:
for file in new_file_name:
    f_read = open('/data/liufengyuan/NLPinFinance/Unziped_Filtered_Data/' + file, 'rb')
    All_data = pickle.load(f_read)
    f_read.close()

    new_file = []

    for idx, item in enumerate(All_data):

        tot_cap_len = len(item['title'])

        if tot_cap_len < file_dict[file[:-4]]['title']:
            tot_fea_len = 0
            for fea in item['feature']:
                tot_fea_len += len(fea)
            
            if tot_fea_len < file_dict[file[:-4]]['feature']:
                tot_des_len = 0
                for des in item['description']:
                    tot_des_len += len(des)

                if tot_des_len < file_dict[file[:-4]]['description']:
                    new_file.append(item)
    print('Save file: ' + file[:-4])
    json.dump(new_file, open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + file[:-4] + '.json', 'w'))

Save file: Clothing_Shoes_and_Jewelry
Save file: Home_and_Kitchen
Save file: Electronics
Save file: Automotive
Save file: Sports_and_Outdoors


### Match data to images

In [51]:
train = os.listdir('/data/liufengyuan/NLPinFinance/trainvalImage/train')
test = os.listdir('/data/liufengyuan/NLPinFinance/trainvalImage/test')
val = os.listdir('/data/liufengyuan/NLPinFinance/trainvalImage/val')

def select_data(item, new_data_val, new_data_test, new_data_train):
    id = item['asin'] + '.jpg'
    if id in val:
        new_data_val.append(item)
    elif id in test:
        new_data_test.append(item)
    elif id in train:
        new_data_train.append(item)

In [52]:
for file in new_file_name:
    f_save = open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + file[:-4] + '.json', 'r')
    total_data = json.load(f_save)
    f_save.close()

    manager = mp.Manager()
    new_data_val = manager.list()
    new_data_test = manager.list()
    new_data_train = manager.list()

    total_data_length = len(total_data)
    each_step = 50000
    print(f'Total number of round is {math.ceil(total_data_length/each_step)}')
    for i in range(math.ceil(total_data_length/each_step)):
        pool = mp.Pool(48)
        count = 0
        start = each_step*i
        end = each_step*(i+1)
        if end > total_data_length:
            end = total_data_length
        for item in total_data[start:end]:
            count += 1
            if count % 2000 == 0:
                print("[%d/%d] Tokenized the captions." %(count, each_step))
            pool.apply_async(select_data, args = (item, new_data_val, new_data_test, new_data_train))
        print('Close pool')
        pool.close()
        print('Join pool')
        pool.join()
        print(f'Round {i} : new_data_val is {len(new_data_val)}, new_data_test is {len(new_data_test)}, new_data_train is {len(new_data_train)}')

    print('Save new_data_val')
    f_save = open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + file[:-4] + '/' + file[:-4] + '_val.json', 'w')
    json.dump(list(new_data_val), f_save)
    f_save.close()

    print('Save new_data_test')
    f_save = open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + file[:-4] + '/' + file[:-4] + '_test.json', 'w')
    json.dump(list(new_data_test), f_save)
    f_save.close()

    print('Save new_data_train')
    f_save = open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + file[:-4] + '/' + file[:-4] + '_train.json', 'w')
    json.dump(list(new_data_train), f_save)
    f_save.close()

Total number of round is 16
[2000/50000] Tokenized the captions.
[4000/50000] Tokenized the captions.
[6000/50000] Tokenized the captions.
[8000/50000] Tokenized the captions.
[10000/50000] Tokenized the captions.
[12000/50000] Tokenized the captions.
[14000/50000] Tokenized the captions.
[16000/50000] Tokenized the captions.
[18000/50000] Tokenized the captions.
[20000/50000] Tokenized the captions.
[22000/50000] Tokenized the captions.
[24000/50000] Tokenized the captions.
[26000/50000] Tokenized the captions.
[28000/50000] Tokenized the captions.
[30000/50000] Tokenized the captions.
[32000/50000] Tokenized the captions.
[34000/50000] Tokenized the captions.
[36000/50000] Tokenized the captions.
[38000/50000] Tokenized the captions.
[40000/50000] Tokenized the captions.
[42000/50000] Tokenized the captions.
[44000/50000] Tokenized the captions.
[46000/50000] Tokenized the captions.
[48000/50000] Tokenized the captions.
[50000/50000] Tokenized the captions.
Close pool
Join pool
Round

### change train, test, val

In [46]:
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)
    data_train = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/' + this_file + '_train.json', 'r'))
    data_test = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/' + this_file + '_test.json', 'r'))
    data_val = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/' + this_file + '_val.json', 'r'))
    print(len(data_train), len(data_test), len(data_val))


    total_dict = []
    total_dict.extend(data_train)
    total_dict.extend(data_test)
    total_dict.extend(data_val)
    print(len(total_dict))

    shuffle(total_dict)

    num_val_test = 10000
    if len(total_dict) > 300000:
        num_val_test = 15000
    elif len(total_dict) < 150000:
        num_val_test = 5000
    new_data_val = total_dict[:num_val_test]
    new_data_test= total_dict[num_val_test:2*num_val_test]
    new_data_train = total_dict[2*num_val_test:]
    

    print('Save new_data_val')
    f_save = open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/' + this_file + '_val.json', 'w')
    json.dump(new_data_val, f_save)
    f_save.close()

    print('Save new_data_test')
    f_save = open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/' + this_file + '_test.json', 'w')
    json.dump(new_data_test, f_save)
    f_save.close()

    print('Save new_data_train')
    f_save = open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/' + this_file + '_train.json', 'w')
    json.dump(new_data_train, f_save)
    f_save.close()

    print(f"{this_file}'s train: {len(new_data_train)} test: {len(new_data_test)} val: {len(new_data_val)}")

Clothing_Shoes_and_Jewelry
672865 10000 10000
692865
Save new_data_val
Save new_data_test
Save new_data_train
Clothing_Shoes_and_Jewelry's train: 662865 test: 15000 val: 15000
Home_and_Kitchen
177994 10000 10000
197994
Save new_data_val
Save new_data_test
Save new_data_train
Home_and_Kitchen's train: 177994 test: 10000 val: 10000
Electronics
129164 10000 10000
149164
Save new_data_val
Save new_data_test
Save new_data_train
Electronics's train: 139164 test: 5000 val: 5000
Automotive
123460 10000 10000
143460
Save new_data_val
Save new_data_test
Save new_data_train
Automotive's train: 133460 test: 5000 val: 5000
Sports_and_Outdoors
104044 10000 10000
124044
Save new_data_val
Save new_data_test
Save new_data_train
Sports_and_Outdoors's train: 114044 test: 5000 val: 5000


### Convert data formet to COCO

In [3]:
splits = ['train', 'val', 'test']

In [5]:
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)
    for split in splits:
        val_data = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/' + this_file + '_' + split + '.json', 'r'))
        val_dict = {'type': 'caption', 'images':[], 'annotations': []}
        val_len = len(val_data)
        count = 0
        for idx, item in enumerate(val_data):
            if len(item['description']) == 1:
                temp_len = 0
                temp_anno = {}
                temp_anno['image_id'] = item['asin']
                temp_anno['id'] = idx + 3058
                temp_anno['caption'] = item['title']
                temp_anno['feature'] = item['feature']
                temp_anno['description'] = item['description'][0]
                val_dict['annotations'].append(temp_anno)

                temp_img = {}
                temp_img['license'] = random.randint(0, 9)
                temp_img['file_name'] = item['asin'] + '.jpg'
                temp_img['id'] = item['asin']
                val_dict['images'].append(temp_img)
        json.dump(val_dict, open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/annotations/' + 'karpathy_split_' + split + '.json', 'w'))
        print(f"Original length is {val_len}.  Output length is {len(val_dict['images'])}. {len(val_dict['images'])/val_len * 100}%")

Clothing_Shoes_and_Jewelry
Original length is 662865.  Output length is 626923. 94.57777978924818%
Original length is 15000.  Output length is 14186. 94.57333333333334%
Original length is 15000.  Output length is 14211. 94.74000000000001%
Home_and_Kitchen
Original length is 177994.  Output length is 173150. 97.27855995145904%
Original length is 10000.  Output length is 9715. 97.15%
Original length is 10000.  Output length is 9727. 97.27%
Electronics
Original length is 139164.  Output length is 126635. 90.99695323503205%
Original length is 5000.  Output length is 4534. 90.68%
Original length is 5000.  Output length is 4561. 91.22%
Automotive
Original length is 133460.  Output length is 128212. 96.06773565113143%
Original length is 5000.  Output length is 4815. 96.3%
Original length is 5000.  Output length is 4814. 96.28%
Sports_and_Outdoors
Original length is 114044.  Output length is 110069. 96.51450317421346%
Original length is 5000.  Output length is 4806. 96.12%
Original length is 5

### Get features

In [49]:
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)
    for split in splits:
        val_data = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/annotations/karpathy_split_' + split + '.json', 'r'))
        val_len = len(val_data['annotations'])
        total_length = 0
        for item in val_data['annotations']:
            total_length += len(item['description'])
        
        print(f"Average description length of {this_file} in {split} set is {total_length/val_len}")

Clothing_Shoes_and_Jewelry
Average description length of Clothing_Shoes_and_Jewelry in train set is 215.4397924466003
Average description length of Clothing_Shoes_and_Jewelry in val set is 216.6265332017482
Average description length of Clothing_Shoes_and_Jewelry in test set is 213.691295475336
Home_and_Kitchen
Average description length of Home_and_Kitchen in train set is 265.0142881894311
Average description length of Home_and_Kitchen in val set is 265.46392177045806
Average description length of Home_and_Kitchen in test set is 266.53613652719235
Electronics
Average description length of Electronics in train set is 338.43957831563154
Average description length of Electronics in val set is 340.79841199823557
Average description length of Electronics in test set is 338.1394431045823
Automotive
Average description length of Automotive in train set is 284.44951330608677
Average description length of Automotive in val set is 283.4753894080997
Average description length of Automotive in te

In [52]:
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)
    for split in splits:
        val_data = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/annotations/karpathy_split_' + split + '.json', 'r'))
        val_dict = {'type': 'caption', 'images':[], 'annotations': []}
        val_len = len(val_data['annotations'])
        count = 0

        
        for item in val_data['annotations']:
            temp_len = 0
            temp_anno = {}
            temp_anno['image_id'] = item['image_id']
            temp_anno['id'] = item['id']
            temp_anno['caption'] = item['caption']
            temp_anno['description'] = item['description']
            for fea in item['feature']:
                temp_anno['feature'] = fea
                val_dict['annotations'].append(temp_anno)

        val_dict['images'] = val_data['images'].copy()
        
        json.dump(val_dict, open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/feature/annotations/karpathy_split_' + split + '.json', 'w'))
        print(f"Original image length is {len(val_data['images'])}.  Output length is {len(val_dict['images'])}. {len(val_dict['images'])/len(val_data['images']) * 100}%")
        print(f"Original annotation length is {val_len}.  Output length is {len(val_dict['annotations'])}. {len(val_dict['annotations'])/val_len * 100}%")

Clothing_Shoes_and_Jewelry
Original image length is 626923.  Output length is 626923. 100.0%
Original annotation length is 626923.  Output length is 3725292. 594.2184287384575%
Original image length is 14186.  Output length is 14186. 100.0%
Original annotation length is 14186.  Output length is 84294. 594.2055547723107%
Original image length is 14211.  Output length is 14211. 100.0%
Original annotation length is 14211.  Output length is 84771. 596.5167827739076%
Home_and_Kitchen
Original image length is 173150.  Output length is 173150. 100.0%
Original annotation length is 173150.  Output length is 743763. 429.54836846664745%
Original image length is 9715.  Output length is 9715. 100.0%
Original annotation length is 9715.  Output length is 41793. 430.19042717447246%
Original image length is 9727.  Output length is 9727. 100.0%
Original annotation length is 9727.  Output length is 41615. 427.829752236044%
Electronics
Original image length is 126635.  Output length is 126635. 100.0%
Orig

### Tokenize data and join again

In [53]:
def title_tokenize(text):
    # 定义分割符，该变量作为超参数，可以根据情况删减
    filters = ['#', '\$', '@', '&', '!',
                '\t', '\n', '\x97', '\x96', 
                ',', '\(', '\)','\[','\]','/',
                ';']

    # 运用正则进行分词:
    # 去除一些特殊记号：
    text = re.sub("&quot", " ", text, flags=re.S)
    text = re.sub("=", " ", text, flags=re.S)
    text = re.sub(":", " ", text, flags=re.S)
    text = re.sub("\s:\s", " ", text, flags=re.S)
    text = re.sub("&amp", " ", text, flags=re.S)
    text = re.sub("AT&amp;T", "brand ATT", text, flags=re.S)
    text = re.sub("T-Mobile", "brand tmobile", text, flags=re.S)
    text = re.sub("\.+", " ", text, flags=re.S)
    # 将网页内容如<>以及分隔符全部替代为空格
    text = re.sub("<.*?>", " ", text, flags=re.S)
    # 将n't转换为not
    text = re.sub("n't", " not", text, flags=re.S)
    # 删去单独出现的-
    text = re.sub("\s-+\s", " ", text, flags=re.S)
    text = re.sub("\s-+$", " ", text, flags=re.S)
    text = re.sub("^-+\s", " ", text, flags=re.S)
    # 将百分数转化：
    text = re.sub("([0-9]+)%", "\g<1> % ", text, flags=re.S)
    # 删除句号并不影响小数
    text = re.sub("\. ", "  ", text, flags=re.S)
    text = re.sub("\.$", "  ", text, flags=re.S)
    text = re.sub("|".join(filters), "  ", text, flags=re.S)

    # 将*删去：
    text = re.sub(" \*([0-9a-zA-Z]+) ", " \g<1> ", text, flags=re.S)
    text = re.sub("\*([0-9a-zA-Z]+) ", "\g<1> ", text, flags=re.S)
    text = re.sub(" \*([0-9a-zA-Z]+)", " \g<1>", text, flags=re.S)

    text = re.sub(" ([0-9a-zA-Z]+)\* ", " \g<1> ", text, flags=re.S)
    text = re.sub("([0-9a-zA-Z]+)\* ", "\g<1> ", text, flags=re.S)
    text = re.sub(" ([0-9a-zA-Z]+)\*", " \g<1>", text, flags=re.S)

    text = re.sub("\*([0-9a-zA-Z]+)\* ", "\g<1> ", text, flags=re.S)
    text = re.sub(" \*([0-9a-zA-Z]+)\* ", " \g<1> ", text, flags=re.S)  
    text = re.sub(" \*([0-9a-zA-Z]+)\*", " \g<1>", text, flags=re.S)
    
    # 将加号单独列出：
    text = re.sub("(.)\+(.)", "\g<1> + \g<2>", text, flags=re.S)
    # 将不是型号且有-和单词的列出：
    text = re.sub("\s([a-zA-Z]+)-([a-zA-Z]+)\s", " \g<1> \g<2> ", text, flags=re.S)
    text = re.sub("\s([a-zA-Z]+)-([a-zA-Z]+)-([a-zA-Z]+)\s", " \g<1> \g<2> \g<3> ", text, flags=re.S)

    text = re.sub("\s-+([0-9a-zA-Z]+)\s", " \g<1> ", text, flags=re.S)
    text = re.sub("\s([0-9a-zA-Z]+)-+\s", " \g<1> ", text, flags=re.S)
    text = re.sub("\s([0-9a-zA-Z]+)-+$", " \g<1>", text, flags=re.S)
    text = re.sub("\s-+([0-9a-zA-Z]+)", " \g<1>", text, flags=re.S)
    # text = re.sub("-([a-zA-Z]+)\s", "\g<1> ", text, flags=re.S)
    # text = re.sub("\s-([a-zA-Z]+)", " \g<1>", text, flags=re.S)
    # 将分隔符全部替代为空格
    # text = re.sub("|".join(filters), " ", text, flags=re.S)
    # 运用strip()返回移除字符串头尾指定的字符（默认为空格或换行符）生成的新字符串:并转换为小写：
    result = [i.strip().lower() for i in text.split()]
    return result

In [54]:
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)
    for split in splits:
        val_data = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/annotations/karpathy_split_' + split + '.json', 'r'))
        val_dict = {'type': 'caption', 'images':[], 'annotations': []}
        val_len = len(val_data['annotations'])
        print(split)
        
        count = 0
        for item in val_data['annotations']:
            count+=1
            temp_len = 0
            temp_anno = {}
            temp_anno['image_id'] = item['image_id']
            temp_anno['id'] = item['id']

            temp_anno['caption'] = ' '.join(title_tokenize(item['caption']))
            temp_anno['description'] = ' '.join(title_tokenize(item['description']))
            temp_anno['feature'] = item['feature']

            val_dict['annotations'].append(temp_anno)
            if count % 5000 == 0:
                print(f"{this_file}'s {split}: " + "[%d/%d] Tokenized the captions." %(count, val_len))

        val_dict['images'] = val_data['images'].copy()
        
        json.dump(val_dict, open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/tokenized/annotations/karpathy_split_' + split + '.json', 'w'))
        print(f"Original image length is {len(val_data['images'])}.  Output length is {len(val_dict['images'])}. {len(val_dict['images'])/len(val_data['images']) * 100}%")
        print(f"Original annotation length is {val_len}.  Output length is {len(val_dict['annotations'])}. {len(val_dict['annotations'])/val_len * 100}%")

Clothing_Shoes_and_Jewelry
train
Clothing_Shoes_and_Jewelry's train: [5000/626923] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [10000/626923] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [15000/626923] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [20000/626923] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [25000/626923] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [30000/626923] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [35000/626923] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [40000/626923] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [45000/626923] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [50000/626923] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [55000/626923] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [60000/626923] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [65000/626923] Tokenized the cap

In [55]:
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)
    
    for split in splits:
        val_data = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/feature/annotations/karpathy_split_' + split + '.json', 'r'))
        val_dict = {'type': 'caption', 'images':[], 'annotations': []}
        val_len = len(val_data['annotations'])
        
        count = 0
        for item in val_data['annotations']:
            count+=1
            temp_len = 0
            temp_anno = {}
            temp_anno['image_id'] = item['image_id']
            temp_anno['id'] = item['id']

            temp_anno['caption'] = ' '.join(title_tokenize(item['caption']))
            temp_anno['description'] = ' '.join(title_tokenize(item['description']))
            temp_anno['feature'] = ' '.join(title_tokenize(item['feature']))

            val_dict['annotations'].append(temp_anno)
            if count % 10000 == 0:
                print(f"{this_file}'s {split}: " + "[%d/%d] Tokenized the captions." %(count, val_len))

        val_dict['images'] = val_data['images'].copy()
        
        json.dump(val_dict, open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/tokenized/feature/annotations/karpathy_split_' + split + '.json', 'w'))
        print(f"Original image length is {len(val_data['images'])}.  Output length is {len(val_dict['images'])}. {len(val_dict['images'])/len(val_data['images']) * 100}%")
        print(f"Original annotation length is {val_len}.  Output length is {len(val_dict['annotations'])}. {len(val_dict['annotations'])/val_len * 100}%")

Clothing_Shoes_and_Jewelry
Clothing_Shoes_and_Jewelry's train: [10000/3725292] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [20000/3725292] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [30000/3725292] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [40000/3725292] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [50000/3725292] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [60000/3725292] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [70000/3725292] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [80000/3725292] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [90000/3725292] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [100000/3725292] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [110000/3725292] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [120000/3725292] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [130000/3725292] Token

#### Title

In [ ]:
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)
    
    for split in splits:
        val_data = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/annotations/karpathy_split_' + split + '.json', 'r'))
        total_length = 0
        count = 0
        for item in val_data['caption']:
            temp_length = len(item['caption'].split())
            if total_length < temp_length:
                total_length = temp_length
            if temp_length <= 80:
                count += 1
        print(f"{this_file}'s {split} max description length: {total_length}. Less than 80: {count} {count/len(val_data['annotations'])*100}%")

#### Description

In [ ]:
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)
    
    for split in splits:
        val_data = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/annotations/karpathy_split_' + split + '.json', 'r'))
        total_length = 0
        count = 0
        for item in val_data['annotations']:
            temp_length = len(item['description'].split())
            if total_length < temp_length:
                total_length = temp_length
            if temp_length <= 80:
                count += 1
        print(f"{this_file}'s {split} max description length: {total_length}. Less than 80: {count} {count/len(val_data['annotations'])*100}%")

### Rename image names and image/file_name

In [21]:
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)
    for split in splits:
        print(split)
        val_data = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/tokenized/annotations/karpathy_split_' + split + '.json', 'r'))
        val_len = len(val_data['images'])
        count = 0
        for idx, item in enumerate(val_data['images']):
            item['file_name'] = item['file_name'][-14:]
        print('save file')
        json.dump(val_data, open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/tokenized/annotations/karpathy_split_' + split + '.json', 'w'))
        print(f"Original length is {val_len}.  Output length is {len(val_data['images'])}. {len(val_data['images'])/val_len * 100}%")

Clothing_Shoes_and_Jewelry
train
save file
Original length is 626923.  Output length is 626923. 100.0%
val
save file
Original length is 14186.  Output length is 14186. 100.0%
test
save file
Original length is 14211.  Output length is 14211. 100.0%
Home_and_Kitchen
train
save file
Original length is 173150.  Output length is 173150. 100.0%
val
save file
Original length is 9715.  Output length is 9715. 100.0%
test
save file
Original length is 9727.  Output length is 9727. 100.0%
Electronics
train
save file
Original length is 126635.  Output length is 126635. 100.0%
val
save file
Original length is 4534.  Output length is 4534. 100.0%
test
save file
Original length is 4561.  Output length is 4561. 100.0%
Automotive
train
save file
Original length is 128212.  Output length is 128212. 100.0%
val
save file
Original length is 4815.  Output length is 4815. 100.0%
test
save file
Original length is 4814.  Output length is 4814. 100.0%
Sports_and_Outdoors
train
save file
Original length is 110069

In [4]:
path = '/data/liufengyuan/NLPinFinance/LFYdata/resized/LFY_2014/'

In [5]:
image_names = os.listdir(path)

In [6]:
my_image_names = []
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)
    for split in splits:
        print(split)
        val_data = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/tokenized/annotations/karpathy_split_' + split + '.json', 'r'))
        for idx, item in enumerate(val_data['images']):
            my_image_names.append(item['file_name'])

Clothing_Shoes_and_Jewelry
train
val
test
Home_and_Kitchen
train
val
test
Electronics
train
val
test
Automotive
train
val
test
Sports_and_Outdoors
train
val
test


In [7]:
len(image_names)

4216066

In [8]:
def delete_img (image):
    if image not in my_image_names:
        os.remove(path + image)

In [9]:
count = 0
for image in image_names:
    count+=1
    if count % 10000 == 0:
        print("[%d/%d] Tokenized the captions." %(count, len(image_names)))
    pool = mp.Pool(48)
    pool.apply_async(delete_img, args = (image))
print('Close pool')
pool.close()
print('Join pool')
pool.join()

Exception ignored in: <function _releaseLock at 0x7f25e2d0b8b0>
Traceback (most recent call last):
  File "/home/liufengyuan/anaconda3/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


In [6]:
import torch
import os

In [7]:
device_ids = range(  os.cpu_count()/2 )
device_ids

TypeError: 'float' object cannot be interpreted as an integer

In [2]:
temp = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/Automotive/tokenized/annotations/karpathy_split_val.json', 'r'))

In [4]:
temp['annotations'][:3]

[{'image_id': 'B0033YTYKM',
  'id': 3058,
  'caption': '1987-1993 ford mustang duraflex gt500 front bumper cover 1 piece',
  'description': '1987-1993 ford mustang duraflex gt500 front bumper cover 1 piece',
  'feature': ['Hand laid, high quality 6 oz. fiberglass',
   'Signature black finish',
   'Proprietary polymer blend for maximum flexibility',
   'High quality weather resistant aluminum mesh grille (where applicable)',
   'Reduced damage rate up to 75%']},
 {'image_id': 'B007KPV8PE',
  'id': 3059,
  'caption': 'kuryakyn 8851 round shift peg covers for metric bikes',
  'description': 'dress up your factory shift peg instead of replacing it this round rubber lined chrome shift peg is now offered in three sizes to accommodate almost any round shift peg',
  'feature': ['New - Retail', '1-Year Warranty', 'KURYAKYN KU 8851']},
 {'image_id': 'B00ANZWJOW',
  'id': 3060,
  'caption': 'silver tone metal snake head shape gear shift knob grip',
  'description': 'perfect car decoration 100 % b